In [14]:
# !pip uninstall pyphi 
# !git clone -b feature/iit-4.0 --single-branch https://github.com/wmayner/pyphi.git
# !pip install -e pyphi

In [30]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [31]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:80% !important; }</style>"))

In [51]:
from visualize_pyphi import *
from visualize_pyphi import utils
from visualize_pyphi import compute
import pickle as pkl
from tqdm.auto import tqdm
import numpy as np

# 4 nodes linear

In [58]:
# ch3 system creation
net_name = "space_4"

th = 1 / 4
exp = 5

s = 0.8
l = 0.25

weights = np.array(
    [
        [s, l, 0, 0],  # A
        [l, s, l, 0],  # B
        [0, l, s, l],  # C
        [0, 0, l, s],  # D
        #A, B, C, D,
    ]
)

# weights += np.random.random((6,6))/100
# weights = np.abs(weights)

node_labels = ["A", "B", "C", "D"]
mech_func = [f for f in "ssss"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    mu=mu,
    si=si,
    l=ll,
    k=kk,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 0, 0, 0)
subsystem = pyphi.Subsystem(network, state)

  0%|          | 0/16 [00:00<?, ?it/s]

Network saved to: space_4


In [59]:
ces = utils.flatten([[d.cause,d.effect] for d in pyphi.compute.ces(subsystem,parallel=True)])
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))


Computing concepts:  73%|██████████████████████████████████████████████████████████████████████▍                         | 11/15 [00:01<00:00,  8.81it/s]
                                                                                                                                                         

there are 4 possible compositional states


In [60]:
max_ces = max(
    [
        compute.filter_ces_parallel(subsystem, ces, compositional_state, max_relations_k=3)
        for compositional_state in tqdm(all_compositional_states)
    ],
    key=lambda c: c["big phi"],
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done   7 out of  64 | elapsed:    1.5s remaining:   12.1s
[Parallel(n_jobs=120)]: Done  14 out of  64 | elapsed:    2.2s remaining:    7.8s
[Parallel(n_jobs=120)]: Done  21 out of  64 | elapsed:    2.7s remaining:    5.5s
[Parallel(n_jobs=120)]: Done  28 out of  64 | elapsed:    3.1s remaining:    4.0s
[Parallel(n_jobs=120)]: Done  35 out of  64 | elapsed:    3.5s remaining:    2.9s
[Parallel(n_jobs=120)]: Done  42 out of  64 | elapsed:    4.1s remaining:    2.2s
[Parallel(n_jobs=120)]: Done  49 out of  64 | elapsed:    4.3s remaining:    1.3s
[Parallel(n_jobs=120)]: Done  56 out of  64 | elapsed:    5.4s remaining:    0.8s
[Parallel(n_jobs=120)]: Done  64 out of  64 | elapsed:    6.8s finished


  0%|          | 0/64 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done   7 out of  64 | elapsed:    2.0s remaining:   16.1s
[Parallel(n_jobs=120)]: Done  14 out of  64 | elapsed:    2.4s remaining:    8.5s
[Parallel(n_jobs=120)]: Done  21 out of  64 | elapsed:    2.6s remaining:    5.4s
[Parallel(n_jobs=120)]: Done  28 out of  64 | elapsed:    3.2s remaining:    4.1s
[Parallel(n_jobs=120)]: Done  35 out of  64 | elapsed:    4.0s remaining:    3.3s
[Parallel(n_jobs=120)]: Done  42 out of  64 | elapsed:    4.3s remaining:    2.3s
[Parallel(n_jobs=120)]: Done  49 out of  64 | elapsed:    4.8s remaining:    1.5s
[Parallel(n_jobs=120)]: Done  56 out of  64 | elapsed:    6.3s remaining:    0.9s
[Parallel(n_jobs=120)]: Done  64 out of  64 | elapsed:    7.5s finished


  0%|          | 0/64 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done   7 out of  64 | elapsed:    1.7s remaining:   13.5s
[Parallel(n_jobs=120)]: Done  14 out of  64 | elapsed:    2.1s remaining:    7.4s
[Parallel(n_jobs=120)]: Done  21 out of  64 | elapsed:    2.3s remaining:    4.7s
[Parallel(n_jobs=120)]: Done  28 out of  64 | elapsed:    2.8s remaining:    3.6s
[Parallel(n_jobs=120)]: Done  35 out of  64 | elapsed:    3.1s remaining:    2.6s
[Parallel(n_jobs=120)]: Done  42 out of  64 | elapsed:    3.7s remaining:    1.9s
[Parallel(n_jobs=120)]: Done  49 out of  64 | elapsed:    4.2s remaining:    1.3s
[Parallel(n_jobs=120)]: Done  56 out of  64 | elapsed:    4.7s remaining:    0.7s
[Parallel(n_jobs=120)]: Done  64 out of  64 | elapsed:    6.2s finished


  0%|          | 0/64 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done   7 out of  64 | elapsed:    1.5s remaining:   12.2s
[Parallel(n_jobs=120)]: Done  14 out of  64 | elapsed:    2.1s remaining:    7.4s
[Parallel(n_jobs=120)]: Done  21 out of  64 | elapsed:    2.9s remaining:    5.9s
[Parallel(n_jobs=120)]: Done  28 out of  64 | elapsed:    3.2s remaining:    4.2s
[Parallel(n_jobs=120)]: Done  35 out of  64 | elapsed:    3.7s remaining:    3.1s
[Parallel(n_jobs=120)]: Done  42 out of  64 | elapsed:    4.2s remaining:    2.2s
[Parallel(n_jobs=120)]: Done  49 out of  64 | elapsed:    4.4s remaining:    1.3s
[Parallel(n_jobs=120)]: Done  56 out of  64 | elapsed:    5.2s remaining:    0.7s
[Parallel(n_jobs=120)]: Done  64 out of  64 | elapsed:    6.5s finished


In [61]:
max_ces['big phi']

2436.309450487817

In [62]:
utils.sepces2df(max_ces['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.479459
1,A,EFFECT,A,0,0.621540
2,B,CAUSE,B,0,0.595967
3,B,EFFECT,B,0,0.700939
4,C,CAUSE,C,0,0.595967
5,C,EFFECT,C,0,0.700939
6,D,CAUSE,D,0,0.479459
7,D,EFFECT,D,0,0.621540
8,AB,CAUSE,AB,00,0.076379
9,AB,EFFECT,AB,00,0.043447


In [63]:
max_ces['MIP']

(((0,), (1, 2, 3)), (0,), (1, 2, 3), CAUSE)

In [12]:

fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/space',surface_opacity=0.7,)

Computing edges:   0%|          | 0/130 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/456 [00:00<?, ?it/s]

# 4 nodes toroidal

In [74]:
# ch3 system creation
net_name = "space_4"

th = 1 / 4
exp = 5

s = 0.8
l = 0.25

weights = np.array(
    [
        [s, l, 0, l],  # A
        [l, s, l, 0],  # B
        [0, l, s, l],  # C
        [l, 0, l, s],  # D
        #A, B, C, D,
    ]
)

# weights += np.random.random((6,6))/100
# weights = np.abs(weights)

node_labels = ["A", "B", "C", "D"]
mech_func = [f for f in "ssss"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    mu=mu,
    si=si,
    l=ll,
    k=kk,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 0, 0, 0)
subsystem = pyphi.Subsystem(network, state)

  0%|          | 0/16 [00:00<?, ?it/s]

Network saved to: space_4


In [75]:
ces = utils.flatten([[d.cause,d.effect] for d in pyphi.compute.ces(subsystem,parallel=True)])
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))


Computing concepts: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  7.19it/s]
                                                                                                                                                         

there are 1 possible compositional states


In [76]:
max_ces = max(
    [
        compute.filter_ces_parallel(subsystem, ces, compositional_state, max_relations_k=3, n_jobs=200)
        for compositional_state in tqdm(all_compositional_states)
    ],
    key=lambda c: c["big phi"],
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

[Parallel(n_jobs=200)]: Using backend MultiprocessingBackend with 200 concurrent workers.
[Parallel(n_jobs=200)]: Done   2 out of   5 | elapsed:    7.1s remaining:   10.7s
[Parallel(n_jobs=200)]: Done   3 out of   5 | elapsed:    7.2s remaining:    4.8s
[Parallel(n_jobs=200)]: Done   5 out of   5 | elapsed:    7.2s remaining:    0.0s
[Parallel(n_jobs=200)]: Done   5 out of   5 | elapsed:    7.2s finished


In [77]:
max_ces['big phi']

1183.2692733561953

In [78]:
utils.sepces2df(max_ces['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.595967
1,A,EFFECT,A,0,0.700939
2,B,CAUSE,B,0,0.595967
3,B,EFFECT,B,0,0.700939
4,C,CAUSE,C,0,0.595967
5,C,EFFECT,C,0,0.700939
6,D,CAUSE,D,0,0.595967
7,D,EFFECT,D,0,0.700939
8,AB,CAUSE,AB,00,0.064884
9,AB,EFFECT,AB,00,0.081128


In [79]:
max_ces['MIP']

(((0,), (1, 2, 3)), (0,), (1, 2, 3), CAUSE)

In [12]:

fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/space',surface_opacity=0.7,)

Computing edges:   0%|          | 0/130 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/456 [00:00<?, ?it/s]

# 5 nodes linear

In [80]:
# ch3 system creation
net_name = "space_4"

th = 1 / 4
exp = 5

s = 0.8
l = 0.25

weights = np.array(
    [
        [s, l, 0, 0, 0],  # A
        [l, s, l, 0, 0],  # B
        [0, l, s, l, 0],  # C
        [0, 0, l, s, l],  # D
        [0, 0, 0, l, s],  # E
        #A, B, C, D, E
    ]
)

# weights += np.random.random((6,6))/100
# weights = np.abs(weights)

node_labels = ["A", "B", "C", "D", "E"]
mech_func = [f for f in "sssss"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    mu=mu,
    si=si,
    l=ll,
    k=kk,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 0, 0, 0, 0)
subsystem = pyphi.Subsystem(network, state)

  0%|          | 0/32 [00:00<?, ?it/s]

Network saved to: space_4


In [81]:
ces = utils.flatten([[d.cause,d.effect] for d in pyphi.compute.ces(subsystem,parallel=True)])
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))

there are 16 possible compositional states


In [82]:
max_ces = max(
    [
        compute.filter_ces_parallel(subsystem, ces, compositional_state, max_relations_k=3, n_jobs=180)
        for compositional_state in tqdm(all_compositional_states)
    ],
    key=lambda c: c["big phi"],
)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/36864 [00:00<?, ?it/s]

[Parallel(n_jobs=180)]: Using backend MultiprocessingBackend with 180 concurrent workers.
[Parallel(n_jobs=180)]: Done   5 tasks      | elapsed:   17.3s
[Parallel(n_jobs=180)]: Done  32 tasks      | elapsed:   41.4s
[Parallel(n_jobs=180)]: Done  61 tasks      | elapsed:   54.8s
[Parallel(n_jobs=180)]: Done  90 tasks      | elapsed:  1.1min
[Parallel(n_jobs=180)]: Done 121 tasks      | elapsed:  1.4min
[Parallel(n_jobs=180)]: Done 152 tasks      | elapsed:  1.6min
[Parallel(n_jobs=180)]: Done 185 tasks      | elapsed:  1.9min
[Parallel(n_jobs=180)]: Done 218 tasks      | elapsed:  2.0min
[Parallel(n_jobs=180)]: Done 253 tasks      | elapsed:  2.2min
[Parallel(n_jobs=180)]: Done 288 tasks      | elapsed:  2.4min
[Parallel(n_jobs=180)]: Done 325 tasks      | elapsed:  2.6min
[Parallel(n_jobs=180)]: Done 362 tasks      | elapsed:  2.8min
[Parallel(n_jobs=180)]: Done 401 tasks      | elapsed:  3.0min
[Parallel(n_jobs=180)]: Done 440 tasks      | elapsed:  3.2min
[Parallel(n_jobs=180)]: Done

KeyboardInterrupt: 

Process ForkPoolWorker-3751:
Process ForkPoolWorker-3862:
Process ForkPoolWorker-3851:
Process ForkPoolWorker-3786:
Process ForkPoolWorker-3810:
Process ForkPoolWorker-3839:
Process ForkPoolWorker-3833:
Process ForkPoolWorker-3853:
Process ForkPoolWorker-3757:
Process ForkPoolWorker-3717:
Process ForkPoolWorker-3829:
Process ForkPoolWorker-3772:
Process ForkPoolWorker-3799:
Process ForkPoolWorker-3776:
Process ForkPoolWorker-3778:
Process ForkPoolWorker-3779:
Process ForkPoolWorker-3725:
Process ForkPoolWorker-3782:
Process ForkPoolWorker-3760:
Process ForkPoolWorker-3758:
Process ForkPoolWorker-3763:
Process ForkPoolWorker-3759:
Process ForkPoolWorker-3738:
Process ForkPoolWorker-3726:
Process ForkPoolWorker-3801:
Process ForkPoolWorker-3764:
Process ForkPoolWorker-3766:
Process ForkPoolWorker-3852:
Process ForkPoolWorker-3846:
Process ForkPoolWorker-3724:
Process ForkPoolWorker-3843:
Process ForkPoolWorker-3735:
Process ForkPoolWorker-3781:
Process ForkPoolWorker-3720:
Process ForkPo

...

  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(

  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 127, in worker
    put((job, i, result))
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 127, in worker
    put((job, i, result))
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 127, in worker
    put((job, i, result))
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/mgrasso

  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 127, in worker
    put((job, i, result))
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/pool.py", line 127, in worker
    put((job, i, result))
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/site-packages/joblib/pool.py", line 145, in get
    racquire()
  File "/home/mgrasso/minicond

  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/site-packages/joblib/pool.py", line 145, in get
    racquire()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/site-packages/joblib/pool.py", line 145, in get
    racquire()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/site-packages/joblib/pool.py", line 169, in put
    wlock_acquire()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/site-packages/joblib/pool.py", line 145, in get
    racquire()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/site-packages/joblib/pool.py", line 145, in get
    racquire()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/site-packages/joblib/pool.py", line 169, in put
    wlock_acquire()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/site-packages/joblib/pool.py", line 145, in get
    racquire()
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/site-packages/joblib/pool.py", line 169, in put
    wlock_acquire()
  File "/home/mgrasso/miniconda3/

KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[o

In [ ]:
max_ces['big phi']

In [ ]:
utils.sepces2df(max_ces['ces'],subsystem)

In [ ]:
max_ces['MIP']

In [12]:

fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/space',surface_opacity=0.7,)

Computing edges:   0%|          | 0/130 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/456 [00:00<?, ?it/s]